In [474]:
import os
from pathlib import Path
import pandas as pd
import numpy as np

In [475]:
project_dir = "C:\\Users\\pyramidheadshark\\Repos\\Steam-AI-Recommendations"
file_path = os.path.join(project_dir, 'data', 'raw', 'Steam Games 2024.csv')
file_path

'C:\\Users\\pyramidheadshark\\Repos\\Steam-AI-Recommendations\\data\\raw\\Steam Games 2024.csv'

In [476]:
df = pd.read_csv(file_path)

C:\Users\pyramidheadshark\AppData\Local\Temp\ipykernel_23336\2618208116.py:1: DtypeWarning: Columns (0,4,5,6,7,8,17,18,19,20,22,23,24,25,26,27,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [477]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97428 entries, 0 to 97427
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   AppID                       97415 non-null  object
 1   Name                        97412 non-null  object
 2   Release date                97418 non-null  object
 3   Estimated owners            97417 non-null  object
 4   Peak CCU                    97417 non-null  object
 5   Required age                97417 non-null  object
 6   Price                       97416 non-null  object
 7   Discount                    97417 non-null  object
 8   DLC count                   97418 non-null  object
 9   About the game              92537 non-null  object
 10  Supported languages         97410 non-null  object
 11  Full audio languages        97410 non-null  object
 12  Reviews                     10132 non-null  object
 13  Header image                97409 non-null  ob

In [478]:
df.columns

Index(['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU',
       'Required age', 'Price', 'Discount', 'DLC count', 'About the game',
       'Supported languages', 'Full audio languages', 'Reviews',
       'Header image', 'Website', 'Support url', 'Support email', 'Windows',
       'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score',
       'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations',
       'Notes', 'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')

In [479]:
columns_to_drop = ['Discount', 'Reviews', 'Support url', 'Support email', 'Notes']

In [480]:
df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
col_dtypes = {
    'AppID': 'Int32',
    'Name': 'object',
    'Release date': 'datetime64[ns]',
    'Estimated owners': 'Int32',
    'Peak CCU': 'Int64',
    'Required age': 'Int8',
    'Price': 'float64',
    'DLC count': 'Int16',
    'About the game': 'object',
    'Supported languages': 'object',
    'Full audio languages': 'object',
    'Header image': 'object',
    'Website': 'object',
    'Windows': 'boolean',
    'Mac': 'boolean',
    'Linux': 'boolean',
    'Metacritic score': 'Int8',
    'Metacritic url': 'object',
    'User score': 'float64',
    'Positive': 'Int32',
    'Negative': 'Int32',
    'Score rank': 'Int8',
    'Achievements': 'Int16',
    'Recommendations': 'Int32',
    'Average playtime forever': 'float64',
    'Average playtime two weeks': 'float64',
    'Median playtime forever': 'float64',
    'Median playtime two weeks': 'float64',
    'Developers': 'object',
    'Publishers': 'object',
    'Categories': 'object',
    'Genres': 'object',
    'Tags': 'object',
    'Screenshots': 'object',
    'Movies': 'object'
}

In [482]:
df = df[pd.to_numeric(df['AppID'], errors='coerce').notnull()]
df['AppID'] = pd.to_numeric(df['AppID'], errors='coerce').astype('Int32')

In [483]:
for col in ['Windows', 'Mac', 'Linux']:
        df[col] = df[col].astype(str).str.lower() == 'true'

In [484]:
try:
    df['Release date'] = pd.to_datetime(df['Release date'], format='%b %d, %Y', errors='coerce')
except ValueError as e:
    print(f"Error converting 'Release date' column: {e}.  Check for inconsistencies in date format.")

In [485]:
valid_estimated_owner_ranges = ['0 - 20000', '50000 - 100000', '20000 - 50000', '200000 - 500000', '100000 - 200000',
                    '2000000 - 5000000', '0 - 0', '500000 - 1000000', '1000000 - 2000000',
                    '20000000 - 50000000', '5000000 - 10000000', '10000000 - 20000000',
                    '50000000 - 100000000', '100000000 - 200000000']

In [486]:
df = df[df['Estimated owners'].isin(valid_estimated_owner_ranges)]

In [487]:
try:
    lower_bounds = df['Estimated owners'].str.split(' - ').str[0].astype(float)
    upper_bounds = df['Estimated owners'].str.split(' - ').str[1].astype(float)
    df['Estimated owners'] = (lower_bounds + upper_bounds) / 2
    df['Estimated owners'] = df['Estimated owners'].astype('Int32')
except (ValueError, AttributeError, IndexError) as e:
    print(f"Error processing 'Estimated owners' column (after filtering): {e}")

In [488]:
try:
    df['Peak CCU'] = pd.to_numeric(df['Peak CCU'], errors='coerce').astype('Int32')
except (ValueError, TypeError) as e:
    print(f"Error converting 'Peak CCU' column: {e}.  Attempting more robust cleaning.")
    df['Peak CCU'] = df['Peak CCU'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df['Peak CCU'] = pd.to_numeric(df['Peak CCU'], errors='coerce').astype('Int32')


In [489]:
valid_ages = [0, 17, 18, 10, 13, 16, 6, 14, 12, 15, 3, 21, 7, 5, 20, 1, 19, 9, 11]

In [490]:
try:
    df['Required age'] = pd.to_numeric(df['Required age'], errors='coerce')
    df = df[df['Required age'].isin(valid_ages)]
    df['Required age'] = df['Required age'].astype('Int8')
except (ValueError, TypeError) as e:
    print(f"Error converting 'Required age' column: {e}.  Invalid values found.")


In [491]:
try:
    df['DLC count'] = df['DLC count'].astype('Int16')
except (ValueError, TypeError) as e:
    print(f"Error converting 'DLC count' column: {e}. Cleaning string values.")

    df['DLC count'] = df['DLC count'].astype(str).str.lower()
    df['DLC count'] = df['DLC count'].str.replace(r'[^\d.]', '', regex=True)
    df['DLC count'] = df['DLC count'].replace(['n/a', 'none', ''], np.nan)

    df['DLC count'] = pd.to_numeric(df['DLC count'], errors='coerce').astype('Int16')


Error converting 'DLC count' column: cannot safely cast non-equivalent object to int16. Cleaning string values.


In [492]:
try:
    df['Metacritic score'] = df['Metacritic score'].replace('<NA>', np.nan)
    df['Metacritic score'] = pd.to_numeric(df['Metacritic score'], errors='coerce').astype('Int8')

except (ValueError, TypeError) as e:
    print(f"Error converting 'Metacritic score' column: {e}.  Check for unexpected values.")

In [493]:
try:
    df['Positive'] = df['Positive'].replace('<NA>', np.nan)
    df['Positive'] = df['Positive'].round().astype('Int32')

except (ValueError, TypeError) as e:
    print(f"Error converting 'Positive' column: {e}. Cleaning non-numeric values.")
    df['Positive'] = df['Positive'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df['Positive'] = pd.to_numeric(df['Positive'], errors='coerce').astype('Int32')

Error converting 'Positive' column: cannot safely cast non-equivalent object to int32. Cleaning non-numeric values.


In [494]:
try:
    df['Negative'] = df['Negative'].replace('<NA>', np.nan)
    df['Negative'] = df['Negative'].round().astype('Int32')

except (ValueError, TypeError) as e:
    print(f"Error converting 'Negative' column: {e}. Cleaning non-numeric values.")
    df['Negative'] = df['Negative'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df['Negative'] = pd.to_numeric(df['Negative'], errors='coerce').astype('Int32')

Error converting 'Negative' column: cannot safely cast non-equivalent object to int32. Cleaning non-numeric values.


In [495]:
try:
    df['Score rank'] = df['Score rank'].replace('<NA>', np.nan)
    df['Score rank'] = df['Positive'].round().astype('Int8')

except (ValueError, TypeError) as e:
    print(f"Error converting 'Score rank' column: {e}. Cleaning non-numeric values.")

    df['Score rank'] = df['Score rank'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df['Score rank'] = df['Score rank'].astype('Int8')

In [496]:
df['Achievements'] = df['Achievements'].replace(['<NA>', ''], np.nan)
df['Achievements'] = pd.to_numeric(df['Achievements'], errors='coerce').astype('Int16')

In [497]:
try:
    df['Recommendations'] = df['Recommendations'].replace('<NA>', np.nan)
    df['Recommendations'] = df['Recommendations'].round().astype('Int32')

except (ValueError, TypeError) as e:
    print(f"Error converting 'Recommendations' column: {e}. Cleaning non-numeric values.")
    df['Recommendations'] = df['Recommendations'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df['Recommendations'] = pd.to_numeric(df['Recommendations'], errors='coerce').astype('Int32')

Error converting 'Recommendations' column: cannot safely cast non-equivalent object to int32. Cleaning non-numeric values.


In [498]:
for col, dtype in col_dtypes.items():
    if col.lower() not in [
        'appid', 'windows', 'mac', 'linux',
        'release date', 'estimated owners', 'required age',
        'dlc count', 'metacritic score', 'positive',
        'negative', 'score rank', 'achievements',
        'recommendations'
        ]:
        try:
            df[col] = df[col].astype(dtype, errors='raise')
        except (ValueError, TypeError) as e:
            print(f"Error converting column '{col}' to type '{dtype}': {e}")
            if dtype in ['Int64', 'int64']:
                df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
            elif dtype == 'float64':
                df[col] = pd.to_numeric(df[col], errors='coerce')
            else:
                print(f"Fallback to object dtype. Column {col} not converted.")

In [499]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97410 entries, 0 to 97427
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   AppID                       97410 non-null  Int32         
 1   Name                        97404 non-null  object        
 2   Release date                97279 non-null  datetime64[ns]
 3   Estimated owners            97410 non-null  Int32         
 4   Peak CCU                    97410 non-null  Int64         
 5   Required age                97410 non-null  Int8          
 6   Price                       97410 non-null  float64       
 7   DLC count                   97410 non-null  Int16         
 8   About the game              92529 non-null  object        
 9   Supported languages         97402 non-null  object        
 10  Full audio languages        97402 non-null  object        
 11  Header image                97402 non-null  object        
